# Activity 3
### Coordinates conversion and repojection
In this activity you will convert map coordinates between different coordinate systems, and change coordinates in a dataset from one coordinate system to another. 

#### Convert from UTM to unprojected or geographic coordintes (lat, lon)

In [6]:
# install utm
import sys
!{sys.executable} -m pip install utm

# import utm
import utm

# the UTM coordinates for Albury
y = 493258.55
x = 6006785.06

# UTM zone and band for Albury
zone = 55
band = 'H'

# print the result
print(utm.to_latlon(y, x, zone, band))

# (-36.08352215209873, 146.92512203814596)

(-36.08352215209873, 146.92512203814596)


 Now let's convert geographic coordinates to UTM.
 To do this, just pass the latitude and longitude to the from_latlon() method, which returns a tuple with the same parameters
that are accepted by the to_latlon() method:

In [7]:
import utm
utm.from_latlon(-36.08352215209873, 146.92512203814596)
 
# (493258.5500000013, 6006785.06000595, 55, 'H')

(493258.5500000013, 6006785.06000595, 55, 'H')

#### Convert map coordinates in degree, minutes, seconds (DMS) to decimal degree, and vice versa
In this task, we'll create two functions that can convert either format into the other

In [8]:
# First, we import the math module to do conversions and the re regular expression module to parse the coordinate string

import math
import re

In [25]:
# user-defined function to convert decimal degrees into a degrees, minutes, and seconds string

def dd2dms(lat, lon):
    """Convert decimal degrees to degrees, minutes, seconds"""
    latf, latn = math.modf(lat)
    lonf, lonn = math.modf(lon)
    latd = int(latn)
    latm = int(latf * 60)
    lats = (lat - latd - latm / 60) * 3600.00
    lond = int(lonn)
    lonm = int(lonf * 60)
    lons = (lon - lond - lonm / 60) * 3600.00
    compass = {'lat': ('N','S'),'lon': ('E','W')}
    lat_compass = compass['lat'][0 if latd >= 0 else 1]
    lon_compass = compass['lon'][0 if lond >= 0 else 1]
    return '{}º {}\' {:.2f}" {}, {}º {}\' {:.2f}"{}'.format(abs(latd),
    abs(latm), abs(lats), lat_compass, abs(lond),
    abs(lonm), abs(lons), lon_compass)

    
    

In [26]:
# user-defined function to convert degrees, minutes, and seconds into decimal degree

def dms2dd(lat, lon):
    lat_deg, lat_min, \
    lat_sec, lat_dir = re.split('[^\d\.A-Z]+', lat)
    lon_deg, lon_min, \
    lon_sec, lon_dir = re.split('[^\d\.A-Z]+', lon)
    lat_dd = float(lat_deg) +\
    float(lat_min)/60 + float(lat_sec)/(60*60);
    lon_dd = float(lon_deg) +\
    float(lon_min)/60 + float(lon_sec)/(60*60);
    if lat_dir == 'S':
        lat_dd *= -1
    if lon_dir == 'W':
        lon_dd *= -1
    return (lat_dd, lon_dd);

In [27]:
# convert decimal degrees into DMS
print(dd2dms(-36.08, 146.92))

# 36º 4' 48.00" S, 146º 55' 12.00"E

36º 4' 48.00" S, 146º 55' 12.00"E


In [30]:
# convert DMS to decimal degree

dms2dd("""36º 4' 48.00" S""", """146º 55' 12.00"E""") # Note that, because the DMS coordinates contain both single and double quotes to represent
# minutes and seconds, we have to use the Python string convention of using triple quotes on each latitude and longitude coordinate to contain both types of quotes so that they are parsed correctly.

# (-36.080000000000005, 146.92)

(-36.080000000000005, 146.92)

### Reproject map coordinates
In GIS, reprojection is all about changing the coordinates in a dataset from one coordinate system to another. Sometimes you need to reproject a shapefile; for a full reprojection, we need some help from the OGR Python API. The OGR API contained in the osgeo module also provides the Open Spatial Reference module, also known as osr, which we'll use for reprojection.

In this task, we'll use a point shapefile containing tree location and height for the Council City of Ryde, NSW. This data was collected in 2013. The Council provides local government services to the suburbs of Ryde, Eastwood, Denistone, Gladesville, Marsfield, Meadowbank, Tennyson Point, and Putney. The point locations are in tranverse mercator projection; specifically, GDA1994 MGA Zone 56. We'll reproject this to WGS84 geographic (or un-project, it rather). You can download this zipped shapefile at https:/ / git. io/ vLbT4.

#### summary of how this has been conducted:
The geometry is transformed and then written to the new file, but the .dbf file is simply copied to the new name as we aren't
changing it. The standard Python shutil module, short for shell utilities, is used to copy .dbf. The source and target shapefile names are variables at the beginning of the script. The target projection is also near the top, which is set using an EPSG code. The script assumes there is a .prj projection file, which defines the source projection. If not, you could manually define it using the same syntax as the target projection. We'll walk through projecting a dataset step by step. Each section is marked with comments.

In [31]:
# import required libraries
from osgeo import ogr
from osgeo import osr
import os
import shutil

In [36]:
# Next, we define the shapefile names as variables
srcName = 'public-trees-2013.shp'
tgtName = 'public-trees-2013_GEO.shp'

In [37]:
# Now, create target spatial reference using the osr module as EPSG code 4326, which is WGS84 Geographic
tgt_spatRef = osr.SpatialReference()
tgt_spatRef.ImportFromEPSG(4326)

0

In [38]:
# Set up our shapefile Reader object using ogr and get the spatial reference

driver = ogr.GetDriverByName('ESRI Shapefile')
src = driver.Open(srcName, 0)
srcLyr = src.GetLayer()
src_spatRef = srcLyr.GetSpatialRef()

In [40]:
# Next, check whether the target shapefile already exists from a previous test run and delete it if it does.

if os.path.exists(tgtName):
    driver.DeleteDataSource(tgtName)

In [41]:
# Now, we can begin building our target layer for the shapefile
tgt = driver.CreateDataSource(tgtName)
lyrName = os.path.splitext(tgtName)[0]
# Use well-known binary format (WKB) to specify geometry
tgtLyr = tgt.CreateLayer(lyrName, geom_type=ogr.wkbPoint)
featDef = srcLyr.GetLayerDefn()
trans = osr.CoordinateTransformation(src_spatRef, tgt_spatRef)

In [47]:
# Next, loop through the features in our source shapefile, reproject them using the Transform() method, and add them to the new shapefile
srcFeat = srcLyr.GetNextFeature()
while srcFeat:
    geom=srcFeat.GetGeometryRef()
    geom.Transform(trans)
    feature = ogr.Feature(featDef)
    feature.SetGeometry(geom)
    tgtLyr.CreateFeature(feature)
    feature.Destroy()
    srcFeat.Destroy()
    srcFeat = srcLyr.GetNextFeature()
src.Destroy
tgt.Destroy

<bound method DataSource.Destroy of <osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x000001800AF51ED0> >>

In [48]:
# Then, we need to create a shapefile .prj file containing projection information as 
# a shapefile has no inherent way to store it

# Convert geometry to Esri flavor of Well-Known Text (WKT) format
# for export to the projection (prj) file.
tgt_spatRef.MorphToESRI()
prj = open(lyrName + '.prj', 'w')
prj.write(tgt_spatRef.ExportToWkt())
prj.close()

In [49]:
# Finally, we can just make a copy of the .dbf source with the new filename as the 
# attributes are part of the reprojection process

srcDbf = os.path.splitext(srcName)[0] + '.dbf'
tgtDbf = lyrName + '.dbf'
shutil.copyfile(srcDbf, tgtDbf)

'public-trees-2013_GEO.dbf'